In [ ]:
import sys
from pathlib import Path

ROOT = Path.cwd()
if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT))
else:
    sys.path.insert(0, str(ROOT.parent))


# 03_surface_and_bespoke

Build a base correlation surface and price bespoke tranches.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from src.interpolation import build_rho_surface
from src.curves import Curve
from src.pricer_tranche import price_tranche_lhp

basecorr_by_tenor = {
    5.0: {0.03: 0.2, 0.07: 0.25, 0.1: 0.3},
    7.0: {0.03: 0.22, 0.07: 0.27, 0.1: 0.32},
}

rho_surface = build_rho_surface(basecorr_by_tenor)
curve = Curve(times=[5.0, 7.0], hazard=[0.02, 0.025])
recovery = 0.4


In [ ]:
tenors = np.linspace(5.0, 7.0, 10)
dets = np.linspace(0.03, 0.1, 10)

Z = np.array([[rho_surface(t, k) for k in dets] for t in tenors])
plt.imshow(Z, extent=[dets.min(), dets.max(), tenors.min(), tenors.max()], aspect="auto", origin="lower")
plt.colorbar(label="rho")
plt.xlabel("Detachment")
plt.ylabel("Tenor")
plt.title("Base correlation surface")
plt.show()


In [ ]:
# Bespoke pricing example
T = 6.0
K1, K2 = 0.03, 0.08
rho = rho_surface(T, K2)
tranche_pv = price_tranche_lhp(T, K1, K2, rho, curve, recovery).pv
print(f"Bespoke PV: {tranche_pv:.6f}")
